In [1]:
import pandas as pd, country_converter as coco

In [6]:
# Open .xlsx file
df = pd.read_excel('vat_substandard_rates.xlsx', sheet_name='VAT rates', skiprows=3)

# Remove bottom 11 rows
df = df[:-11].copy()

# Rename 'Unnamed: 0' to 'Country'
df.rename(columns={'Unnamed: 0': 'Country'}, inplace=True)

# Add 'ISO3' column
df['ISO3'] = coco.convert(names=df['Country'], to='ISO3')

df['ISO - numeric'] = coco.convert(names=df['Country'], to='UN')

# Add continent column
df['Continent'] = coco.convert(names=df['Country'], to='continent')

# Drop if not found
df = df[df['ISO3'] != 'not found'].copy()

# Filter to Continent
df = df[df['Continent'] == 'Europe'].reset_index(drop=True).copy()

Channel Islands not found in regex
Channel Islands not found in regex
Channel Islands not found in regex


In [7]:
# Clean `VAT/GST (standard)` column. Rules:
# If value is percentage (e.g. 20%), convert to decimal (e.g. 0.20)
# If value us 'no', convert to 0
# If value starts with percentage, then has other text, take the first percentage and convert to decimal
# If value is NaN, convert to 0

def clean_vat(row):
    if row['VAT/GST (standard)'] == 'no':
        return 0
    elif pd.isna(row['VAT/GST (standard)']):
        return 0
    else:
        # Convert value to string and split
        if '%' in str(row['VAT/GST (standard)']):
            value_string = str(row['VAT/GST (standard)']).split('%')[0]
            return float(value_string) / 100
        else:
            return row['VAT/GST (standard)']

df['vat_cleaned'] = df.apply(clean_vat, axis=1)

In [8]:
df.to_csv('imf_vat_europe.csv', index=False)

In [4]:
df

,Country,Income group,Region,Country groups,VAT/GST (standard),VAT/GST (reduced),VAT/GST (increased),ISO3,Continent,id
0,Afghanistan,Low income,Southern Asia,MENAP,no,no,no,AFG,Asia,4.0
1,Albania,Upper middle income,Southern Europe,NaN,20%,"0%, 6%",no,ALB,Europe,8.0
2,Algeria,Upper middle income,Northern Africa,MENAP,0.19,"0%, 9%",no,DZA,Africa,12.0
3,American Samoa,Upper middle income,Polynesia,NaN,NaN,NaN,NaN,ASM,Oceania,16.0
4,Andorra,High income: nonOECD,Southern Europe,NaN,0.045,"0%, 1% and 2.5%",9.5% (for banking and financial services),AND,Europe,20.0
...,...,...,...,...,...,...,...,...,...,...
213,Venezuela,High income: nonOECD,South America,CELAC,0.16,"0%, 8%",0.31,VEN,America,862.0
214,Vietnam,Lower middle income,South-Eastern Asia,ASEAN,"10%\n\n0%, 2%, 3% or 5% of the revenue for for...","0%, 5%",no,VNM,Asia,704.0
215,Yemen,Lower middle income,Middle East,MENAP,5%,0% (export of goods and services and internati...,"10%, 20%, 90%",YEM,Asia,887.0
216,Zambia,Lower middle income,Eastern Africa,SADC,16%,0%,no,ZMB,Africa,894.0


Let's combine with TradingEconomics dataset on standard sales tax / vat rates.

In [21]:
# Open .xlsx file
df_te = pd.read_excel('vat_substandard_rates.xlsx', sheet_name='TE')

In [22]:
df_te

,Country,Last,Reference
0,Afghanistan,10.0,2022-12-01
1,Albania,20.0,2023-12-01
2,Algeria,19.0,2023-12-01
3,Andorra,4.5,2022-12-01
4,Angola,14.0,2023-12-01
...,...,...,...
174,Vanuatu,15.0,2023-12-01
175,Venezuela,16.0,2023-12-01
176,Vietnam,10.0,2023-12-01
177,Zambia,16.0,2023-12-01
